<a href="https://colab.research.google.com/github/gorogoro-uk/TensorFlowLearning/blob/master/TensorFlow_in_Practice_Course1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TensorFlow in Practice course 1


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
# 1. House Price Prediction
##############################
# a) Basic Flow of model usage
#############################


import tensorflow as tf
import numpy as np
from tensorflow import keras

# data
xs = np.array([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], dtype=float)
ys = np.array([1.0, 1.5, 2.0, 2.5, 3.0, 3.5], dtype=float)

# model definition
model = tf.keras.Sequential([keras.layers.Dense(units=1, input_shape=[1])])

# compile with loss and optimizer
model.compile(optimizer='sgd', loss='mean_squared_error')

# fit model
model.fit(xs, ys, epochs=1000)

# predict 
print(model.predict([7.0]))

Epoch 1/1000
1/1 [==============================] - 0s 1ms/step - loss: 24.5241
Epoch 2/1000
1/1 [==============================] - 0s 1ms/step - loss: 11.3554
Epoch 3/1000
1/1 [==============================] - 0s 1ms/step - loss: 5.2604
Epoch 4/1000
1/1 [==============================] - 0s 2ms/step - loss: 2.4395
Epoch 5/1000
1/1 [==============================] - 0s 3ms/step - loss: 1.1338
Epoch 6/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.5295
Epoch 7/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.2497
Epoch 8/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.1202
Epoch 9/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0603
Epoch 10/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0325
Epoch 11/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0196
Epoch 12/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.0136
Epoch 13/1000
1/1 [============================

In [ ]:
# 2. Handwriting Recognition
##############################
# a) Callback: used to terminate learning based on desired accuracy level
# b) TensorFlow MINST dataset
# c) Normalising pixel values
# d) Mulitlayer Dense network with softmax
#############################


import tensorflow as tf

# callback to stop learning after attaining desired accuracy
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.99):
      print("\nReached 99% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

# TensorFlow dataset - MINST
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # normalising data

# model definition
# simple Dense (fully connected) layers
# softmax for multi-class outcome
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# model fit
model.fit(x_train, y_train, epochs=10, callbacks=[callbacks])

11493376/11490434 [==============================] - 1s 0us/step
Epoch 1/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1989 - accuracy: 0.9412
Epoch 2/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0808 - accuracy: 0.9753
Epoch 3/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0521 - accuracy: 0.9839
Epoch 4/10
1875/1875 [==============================] - 8s 5ms/step - loss: 0.0356 - accuracy: 0.9886
Epoch 5/10
1874/1875 [============================>.] - ETA: 0s - loss: 0.0276 - accuracy: 0.9911
Reached 99% accuracy so cancelling training!
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0276 - accuracy: 0.9911


In [ ]:
# 3. MINST with Convolutions
##############################
# a) Accuracy Callback
# b) TensorFlow MINST dataset
# c) Reshaping images 
# d) Convolution & MaxPooling
#############################

import tensorflow as tf

# accuracy callback
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.998):
      print("\nReached 99.8% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

# TensorFlow MINST dataset
mnist = tf.keras.datasets.mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images.reshape(60000, 28, 28, 1)
training_images=training_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images=test_images/255.0

# model definition
# convolution & max-pooling layers
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# fit model
model.fit(training_images, training_labels, epochs=10, callbacks=[callbacks])

Epoch 1/10
1875/1875 [==============================] - 36s 19ms/step - loss: 0.1562 - accuracy: 0.9540
Epoch 2/10
1875/1875 [==============================] - 36s 19ms/step - loss: 0.0526 - accuracy: 0.9841
Epoch 3/10
1875/1875 [==============================] - 35s 19ms/step - loss: 0.0350 - accuracy: 0.9885
Epoch 4/10
1875/1875 [==============================] - 35s 19ms/step - loss: 0.0213 - accuracy: 0.9931
Epoch 5/10
1875/1875 [==============================] - 35s 19ms/step - loss: 0.0160 - accuracy: 0.9947
Epoch 6/10
1875/1875 [==============================] - 35s 19ms/step - loss: 0.0114 - accuracy: 0.9962
Epoch 7/10
1875/1875 [==============================] - 35s 19ms/step - loss: 0.0082 - accuracy: 0.9973
Epoch 8/10
1875/1875 [==============================] - 36s 19ms/step - loss: 0.0067 - accuracy: 0.9978
Epoch 9/10
1874/1875 [============================>.] - ETA: 0s - loss: 0.0054 - accuracy: 0.9983
Reached 99.8% accuracy so cancelling training!
1875/1875 [============

In [ ]:
# 4. Complex Images
##############################
# a) Data taken from external website
# b) Accuracy Callback
# c) Mulitple Convolution & MaxPooling layers
# d) DataGenerator - flow from directory
#############################


import tensorflow as tf
import os
import zipfile

DESIRED_ACCURACY = 0.999

# get dataset from Laurence's site
!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

# unzip data
zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

# accuracy callback
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>DESIRED_ACCURACY):
      print("\nReached 99.9% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

# model definition
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])


# dataflow generator
# flow from directory - automatically create binary labels based on directory
# load in batches of 10
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
        "/tmp/h-or-s",  
        target_size=(150, 150), 
        batch_size=10,
        class_mode='binary')

# fit model
history = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1,
      callbacks=[callbacks])

--2020-07-01 16:31:21--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.112.128, 74.125.124.128, 172.217.212.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.112.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.01s   

2020-07-01 16:31:21 (180 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]

Found 80 images belonging to 2 classes.
Epoch 1/15
8/8 [==============================] - 2s 203ms/step - loss: 0.8262 - accuracy: 0.6250
Epoch 2/15
8/8 [==============================] - 2s 200ms/step - loss: 0.3775 - accuracy: 0.9000
Epoch 3/15
8/8 [==============================] - 2s 198ms/step - loss: 0.1776 - accuracy: 0.9375
Epoch 4/15
8/8 [==============================] - 2s 199ms/st